In [1]:
%%spark

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1561331732791_0001,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.


In [2]:
df = spark.read.csv("s3a://billingarun/Billing.csv")

VBox()

In [3]:
df.printSchema()
df.show(5)

VBox()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)

+---------+--------------+--------------------+--------+------------+------------+------------+-------+
|      _c0|           _c1|                 _c2|     _c3|         _c4|         _c5|         _c6|    _c7|
+---------+--------------+--------------------+--------+------------+------------+------------+-------+
|Zone Name| Main Category|        Sub Category|    Year|        Sale|      Demand|  Collection|EFFI(%)|
|PUNE ZONE| HT-INDUSTRIAL|    HT-IND.(EXPRESS)|FY 12-13| 1,253.5944 | 1,020.3831 |   967.3706 |105.48%|
|PUNE ZONE|P.D. CONSUMERS|     HT-PD CONSUMERS|FY 12-13|        -   |        -   |     0.0692 |  0.00%|
|PUNE ZONE| HT-INDUSTRIAL|HT-IND.(NON-EXPRESS)|FY 12-13| 1,508.0498 | 1,131.4004 | 1,101.6

In [4]:
df = spark.read.csv("s3a://billingarun/Billing.csv",sep=",", inferSchema=True, header=True)

VBox()

In [5]:
df.printSchema()
df.show(5)

VBox()

root
 |-- Zone Name: string (nullable = true)
 |-- Main Category: string (nullable = true)
 |-- Sub Category: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Sale: string (nullable = true)
 |-- Demand: string (nullable = true)
 |-- Collection: string (nullable = true)
 |-- EFFI(%): string (nullable = true)

+---------+--------------+--------------------+--------+------------+------------+------------+-------+
|Zone Name| Main Category|        Sub Category|    Year|        Sale|      Demand|  Collection|EFFI(%)|
+---------+--------------+--------------------+--------+------------+------------+------------+-------+
|PUNE ZONE| HT-INDUSTRIAL|    HT-IND.(EXPRESS)|FY 12-13| 1,253.5944 | 1,020.3831 |   967.3706 |105.48%|
|PUNE ZONE|P.D. CONSUMERS|     HT-PD CONSUMERS|FY 12-13|        -   |        -   |     0.0692 |  0.00%|
|PUNE ZONE| HT-INDUSTRIAL|HT-IND.(NON-EXPRESS)|FY 12-13| 1,508.0498 | 1,131.4004 | 1,101.6759 |102.70%|
|PUNE ZONE|P.D. CONSUMERS|     HT-PD CONSUMERS|FY

In [6]:
from pyspark.sql.types import StructType as R, StructField as Fld, DoubleType as Dbl, StringType as Str, IntegerType as Int, DateType as Date
paymentSchema = R([
    Fld("Zone Name",Str()),
    Fld("Main Category",Str()),
    Fld("Sub Category",Str()),
    Fld("Year",Str()),
    Fld("Sale",Dbl()),
    Fld("Demand",Dbl()),
    Fld("Collection",Dbl()),
    Fld("EFFI(%)",Dbl()),
])

VBox()

In [7]:
dfPaymentWithSchema = spark.read.csv("s3a://billingarun/Billing.csv",sep=",", schema=paymentSchema, header=True)

VBox()

In [8]:
dfPaymentWithSchema.printSchema()
df.show(5)

VBox()

root
 |-- Zone Name: string (nullable = true)
 |-- Main Category: string (nullable = true)
 |-- Sub Category: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Sale: double (nullable = true)
 |-- Demand: double (nullable = true)
 |-- Collection: double (nullable = true)
 |-- EFFI(%): double (nullable = true)

+---------+--------------+--------------------+--------+------------+------------+------------+-------+
|Zone Name| Main Category|        Sub Category|    Year|        Sale|      Demand|  Collection|EFFI(%)|
+---------+--------------+--------------------+--------+------------+------------+------------+-------+
|PUNE ZONE| HT-INDUSTRIAL|    HT-IND.(EXPRESS)|FY 12-13| 1,253.5944 | 1,020.3831 |   967.3706 |105.48%|
|PUNE ZONE|P.D. CONSUMERS|     HT-PD CONSUMERS|FY 12-13|        -   |        -   |     0.0692 |  0.00%|
|PUNE ZONE| HT-INDUSTRIAL|HT-IND.(NON-EXPRESS)|FY 12-13| 1,508.0498 | 1,131.4004 | 1,101.6759 |102.70%|
|PUNE ZONE|P.D. CONSUMERS|     HT-PD CONSUMERS|FY

In [9]:
dfPaymentWithSchema.createOrReplaceTempView("billing")
spark.sql("""
    SELECT Year as d, sum(Sale) as revenue
    FROM billing
    GROUP by d
    order by revenue desc
""").show()

VBox()

+--------+------------------+
|       d|           revenue|
+--------+------------------+
|FY 12-13|2509.7799999999993|
|FY 13-14|2404.7116000000005|
|    null|              null|
+--------+------------------+